# Kết nối đến drive và import các thư viện cần thiết

In [2]:
!pip install pyvi --quiet
!pip install keras_preprocessing --quiet
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.3 MB/s eta 0:00:00


In [3]:
import os

import re
import pickle
import numpy as np
from pyvi import ViTokenizer
from tensorflow import keras
import tensorflow as tf
from keras_preprocessing.sequence import pad_sequences

In [4]:
# Import Thư viện
from model_transformer import *
from transformers import AutoTokenizer, BertTokenizer

# 2. Xây dựng các hàm để load các file, model đã training và hàm để thử nghiệm

In [5]:
MAX_LENGTH = 60

In [9]:
# saving
def save_file(data, filename):
    with open(filename + '.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
# loading
def load_file(filename):
    with open(filename + '.pickle', 'rb') as handle:
        data = pickle.load(handle)
    return data

def load_model(filename):
    return keras.models.load_model(filename)

def make_clean_text_en(text_to_clean):
    res = text_to_clean.lower()
    res = re.sub(r"i'm", "i am", res)
    res = re.sub(r"he's", "he is", res)
    res = re.sub(r"she's", "she is", res)
    res = re.sub(r"it's", "it is", res)
    res = re.sub(r"that's", "that is", res)
    res = re.sub(r"what's", "what is", res)
    res = re.sub(r"where's", "where is", res)
    res = re.sub(r"how's", "how is", res)
    res = re.sub(r"\'ll", " will", res)
    res = re.sub(r"\'ve", " have", res)
    res = re.sub(r"\'re", " are", res)
    res = re.sub(r"\'d", " would", res)
    res = re.sub(r"\'re", " are", res)
    res = re.sub(r"won't", "will not", res)
    res = re.sub(r"can't", "cannot", res)
    res = re.sub(r"n't", " not", res)
    res = re.sub(r"n'", "ng", res)
    res = re.sub(r"'bout", "about", res)
    res = re.sub(r"'til", "until", res)
    res = re.sub(r'[!“”"#$%&\()*+,-./:;<=>?@[\]^_`{|}~]', "", res)
    res = re.sub(r'\s+', ' ', res).strip()
    return res

In [7]:
tokenizer_vi_transformer = AutoTokenizer.from_pretrained("vinai/phobert-base")
tokenizer_en_transformer = BertTokenizer.from_pretrained("bert-base-cased")

special_tokens = {'cls_token': '<s>', 'sep_token': '</s>'}
tokenizer_en_transformer.add_special_tokens(special_tokens_dict=special_tokens)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2

In [ ]:
config_params = {
    'num_layers': 6,
    'd_model': 128,
    'dff': 512,
    'num_heads': 8,
    'input_vocab_size': tokenizer_en_transformer.vocab_size + 2,
    'target_vocab_size': tokenizer_vi_transformer.vocab_size + 2,
    'dropout_rate': 0.1
}

In [ ]:
learning_rate = CustomSchedule(config_params['d_model'])

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate(2e-5), beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# Create model
transformer = create_model(config_params)

In [ ]:
checkpoint_path = os.path.join('../model_checkpoint/checkpoint_bert')

ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [ ]:
def evaluate(inp_sentence):
    start_token = [tokenizer_en_transformer.cls_token_id] # [tokenizer_en.vocab_size]
    end_token = [tokenizer_en_transformer.sep_token_id] # # [tokenizer_en.vocab_size + 1]
    
    # inp sentence is english, hence adding the start and end token
    inp_sentence = tokenizer_en_transformer.encode(inp_sentence)
    encoder_input = tf.expand_dims(inp_sentence, 0)

    # as the target is vietnamese, the first word to the transformer should be the
    # vietnamese start token.
    ## decoder_input = [tokenizer_vn.vocab_size] # <start>
    decoder_input = [tokenizer_vi_transformer.bos_token_id]
    output = tf.expand_dims(decoder_input, 0)
        
    for i in range(MAX_LENGTH):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)
    
        # predictions.shape == (batch_size, seq_len, vocab_size)
        predictions, attention_weights = transformer(encoder_input, 
                                                    output,
                                                    False,
                                                    enc_padding_mask,
                                                    combined_mask,
                                                    dec_padding_mask)
        
        # select the last word from the seq_len dimension
        predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        
        # return the result if the predicted_id is equal to the end token
        if predicted_id == tokenizer_vi_transformer.eos_token_id:
            return tf.squeeze(output, axis=0), attention_weights
        
        # concatentate the predicted_id to the output which is given to the decoder
        # as its input.
        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights


def translate_transformer(sentence):
    clean_sentence = make_clean_text_en(sentence) # pre-process input
    result, attention_weights = evaluate(clean_sentence)

    predicted_sentence = tokenizer_vi_transformer.decode(result.numpy())
    print('Predicted translation (transformers): {}'.format(predicted_sentence.replace('<s>', '')))

In [ ]:
%%time
translate_transformer("The science behind a climate headline")

Predicted translation (transformers):  đằng sau tiêu_đề khí_hậu
CPU times: user 5.49 s, sys: 1.11 s, total: 6.61 s
Wall time: 10.3 s


# 3. Load model đã training

In [ ]:
ROOT_LSTM = '../model_checkpoint/pre-trained LSTM/'

tokenizer_en_LSTM = load_file(ROOT_LSTM + 'tokenizer_en')
tokenizer_vi_LSTM = load_file(ROOT_LSTM + 'tokenizer_vi')

enc_model = load_model(ROOT_LSTM + 'enc_model.h5')
dec_model = load_model(ROOT_LSTM + 'dec_model.h5')

In [ ]:
def str_to_tokens(sentence):
    words = make_clean_text_en(sentence)
    tokens_list = tokenizer_en_LSTM.texts_to_sequences([words])
    return pad_sequences(tokens_list, maxlen=MAX_LENGTH,padding='post')

def translate_LSTM(sentence):
    states_values = enc_model.predict(str_to_tokens(sentence), verbose=0)
    empty_target_seq = np.zeros((1,1))
    empty_target_seq[0,0] = tokenizer_vi_LSTM.word_index['<s>']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition:
        dec_outputs, h, c = dec_model.predict([empty_target_seq]+states_values, verbose=0)
        sampled_word_index = np.argmax(dec_outputs[0,-1, :])
        sampled_word = None
        for word, index in tokenizer_vi_LSTM.word_index.items():
            if sampled_word_index == index:
                if word != '</s>':
                    decoded_translation += f'{word} '
                sampled_word = word

        if sampled_word == '</s>' or len(decoded_translation.split()) > MAX_LENGTH:
            stop_condition = True
        empty_target_seq = np.zeros((1,1))
        empty_target_seq[0,0] = sampled_word_index
        states_values = [h,c]
    print('Predicted translation (LSTM): {}'.format(decoded_translation))

In [ ]:
%%time
translate_LSTM("The science behind a climate headline")

Predicted translation (LSTM): khoa_học là một trong những khí_hậu đang gây ra 
CPU times: user 5.13 s, sys: 257 ms, total: 5.39 s
Wall time: 7.53 s


# 4.Thử nghiệm

In [ ]:
%%time
translate_LSTM("Thank you very much for your time")

Predicted translation (LSTM): cảm_ơn các bạn đã rất nhiều thời_gian cho tôi 
CPU times: user 680 ms, sys: 19.4 ms, total: 700 ms
Wall time: 692 ms


In [ ]:
%%time
translate_transformer("Thank you very much for your time")

Predicted translation (transformers):  vì cảm_ơn các bạn rất nhiều vì thời_gian
CPU times: user 2.83 s, sys: 16.4 ms, total: 2.84 s
Wall time: 2.89 s


In [ ]:
%%time
translate_LSTM("They are both two branches of the same field")

Predicted translation (LSTM): chúng đều là hai phần của cả hai 
CPU times: user 887 ms, sys: 18.5 ms, total: 905 ms
Wall time: 1.09 s


In [ ]:
%%time
translate_transformer("They are both two branches of the same field")

Predicted translation (transformers):  đều là hai nhánh của cùng một lĩnh_vực
CPU times: user 2.66 s, sys: 25.8 ms, total: 2.69 s
Wall time: 2.71 s


In [ ]:
def translate(sentence):
    translate_transformer(sentence)
    translate_LSTM(sentence)

In [ ]:
translate('and so scientists and engineers when you aposve solved this equation by all means talk nerdy to me')

Predicted translation (transformers):  và các nhà khoa_học và kỹ_sư khi bạn đã giải_quyết được phương_trình này bằng tất_cả có_nghĩa_là quot thần_kinh quot cho tôi
Predicted translation (LSTM): và các nhà khoa_học và các bạn đã phải làm khi tôi nói về điều này vì tất_cả những gì có_thể là từ một câu nói 
